In [68]:
import pandas as pd
import geopandas as gpd

In [111]:
def describir_shp(gdf):
    area_cargado = gdf['area'].sum()
    gdf['area'] = gdf.geometry.area / 10000
    area_recalculado = gdf['area'].sum()
    gdf['idd'] = gdf.apply(lambda row: f"{row['unidad_01']}|{row['unidad_03']}|{row['unidad_05']}", axis=1)
    gdf_disolve = gdf.dissolve(by='idd')
    gdf_disolve['area'] = gdf_disolve.geometry.area / 10000
    area_disolve = gdf_disolve['area'].sum()
    estado_datos = {
        'Items' : ['Area cargado', 'Area recalculado', 'Area disolve'],
        'Datos' : [round(area_cargado, 2), round(area_recalculado, 2), round(area_disolve, 2)]
    }
    return pd.DataFrame(estado_datos)

In [112]:
path_shp_cosecha = r'C:/Documents/OneDrive - Ingenio Azucarero Guabira S.A/INFORMACION GENERAL/CATASTRO/NUEVO/cosecha2024.shp'
path_shp_catastro = r'C:/Documents/OneDrive - Ingenio Azucarero Guabira S.A/INFORMACION GENERAL/CATASTRO/NUEVO/catastro170524.shp'

In [113]:
gdf_cosecha = gpd.read_file(path_shp_cosecha)
gdf_catastro = gpd.read_file(path_shp_catastro)

In [114]:
describir_shp(gdf_cosecha)

,Items,Datos
0,Area cargado,53938.92
1,Area recalculado,53939.09
2,Area disolve,53939.09


In [115]:
gdf_catastro_canha = gdf_catastro[gdf_catastro['cultivo']=='canha'].copy()

In [116]:
describir_shp(gdf_catastro_canha)

,Items,Datos
0,Area cargado,54477.90
1,Area recalculado,54435.22
2,Area disolve,54435.22


In [117]:
def comprobar_idd_repetidos(gdf):
    gdf['idd'] = gdf.apply(lambda row: f"{row['unidad_01']}|{row['unidad_03']}|{row['unidad_05']}", axis=1)
    gdf_orden = gdf.sort_values(by='idd')
    conteos = gdf['idd'].value_counts()
    valores_repetidos = conteos[conteos > 1]
    return valores_repetidos

# COMPROBAR CATASTRO

In [118]:
comprobar_idd_repetidos(gdf_catastro_canha)

idd
377|11207|L1.4     2
1742|10716|L2      2
365|15001|L23.1    2
1035|13660|L4      2
1484|12314|L1.2    2
503|12725|L4       2
2017|2674|L2       2
691|40253|L5       2
898|41790|L9.2     2
349|2161|L1.2      2
288|2588|L3        2
119|7077|L2        2
365|15001|L14.1    2
1425|40132|L4.1    2
1395|13058|L2      2
Name: count, dtype: int64

In [119]:
gdf_catastro_disolve = gdf_catastro_canha.dissolve(by='idd')
gdf_catastro_disolve['area'] = gdf_catastro_disolve.geometry.area / 10000
gdf_catastro_disolve = gdf_catastro_disolve.reset_index()
area_catastro_disolve = gdf_catastro_disolve['area'].sum()

In [120]:
area_catastro_disolve

54435.221193389036

In [121]:
comprobar_idd_repetidos(gdf_catastro_disolve)

Series([], Name: count, dtype: int64)

# COMPROBAR SHP COSECHA

In [122]:
gdf_cosecha_disolve = gdf_cosecha.dissolve(by='idd')
gdf_cosecha_disolve['area'] = gdf_cosecha_disolve.geometry.area / 10000
gdf_cosecha_disolve = gdf_cosecha_disolve.reset_index()
area_cosecha_disolve = gdf_cosecha_disolve['area'].sum()

In [123]:
area_cosecha_disolve

53939.09026723008

In [124]:
comprobar_idd_repetidos(gdf_cosecha_disolve)

Series([], Name: count, dtype: int64)

# COMPARAR CATASTRO - COSECHA

In [126]:
gdf_catastro_disolve['area'].sum()

54435.221193389036

In [127]:
gdf_cosecha_disolve['area'].sum()

53939.09026723008

In [128]:
gdf_catastro_disolve.columns

Index(['idd', 'geometry', 'id', 'unidad_01', 'unidad_02', 'unidad_03',
       'unidad_04', 'unidad_05', 'variedad', 'soca', 'zona', 'textura',
       'cultivo', 'zafra', 'financia', 'fs', 'fc', 'area', 'estado'],
      dtype='object')

In [130]:
gdf_cat_pivot = gdf_catastro_disolve[['idd', 'area']]

In [131]:
gdf_cos_pivot = gdf_cosecha_disolve[['idd', 'area']]

In [135]:
df_cat_merged = pd.merge(gdf_cat_pivot, gdf_cos_pivot, on='idd', how='left')

In [137]:
df_cat_merged.to_excel('merge.xlsx')